## Libraries

In [ ]:
import pyarrow.csv as csv
import pyarrow.parquet as pq 
import glob
import os
import pyarrow as pa
from pyarrow import parquet 
import polars as pl
from datetime import datetime
import numpy as np
import pandas as pd
from plotnine import ggplot, aes, geom_point, theme, scale_y_reverse, scale_color_identity

## General Functions

In [ ]:
# Coordinate parser to fix formatting inconsistencies
def parse_coordinate(coordinate):
    if len(coordinate) < 20:
        return coordinate
    else:
        coordinate = coordinate.replace(" ", "").replace(":", ",").split(",")
        x = coordinate[1]
        y = coordinate[3]
        return f"{x}" + "," + f"{y}"

## Tony Code

In [ ]:
# Import data
data1 = pd.read_csv(r"C:\Users\tony9\Downloads\merged-csv-files1.csv")
data2 = pd.read_csv("C:\\Users\\tony9\\OneDrive\\Documents\\merged-csv-files2.csv")
data3 = pd.read_csv("C:\\Users\\tony9\\OneDrive\\Documents\\merged-csv-files3.csv")
data4 = pd.read_csv("C:\\Users\\tony9\\OneDrive\\Documents\\merged-csv-files4.csv")

data1 = data1.dropna()
data = data1
data

In [ ]:
# Convert time from string to datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])
data['date'] = data['timestamp'].dt.date

In [ ]:
# Drop coordinate and pixel color
new_data = data.drop(['coordinate', 'pixel_color'], axis=1)

In [ ]:
user_frequency = data.groupby('user').size().reset_index(name='frequency')
user_frequency.to_csv("C:/Users/tony9/Downloads/filename1.csv", index=False)

user_frequency2 = data2.groupby('user').size().reset_index(name='frequency')
user_frequency2.to_csv("C:/Users/tony9/Downloads/filename.csv", index=False)

user_frequency3 = data3.groupby('user').size().reset_index(name='frequency')
user_frequency3.to_csv("C:/Users/tony9/Downloads/filename3.csv", index=False)

user_frequency4 = data4.groupby('user').size().reset_index(name='frequency')
user_frequency4.to_csv("C:/Users/tony9/Downloads/filename4.csv", index=False)

In [ ]:
sorted_user_frequency4 = user_frequency4.sort_values(by='frequency', ascending=False)
sorted_user_frequency4

In [ ]:
merged_df = pd.concat([user_frequency, user_frequency2, user_frequency3, user_frequency4])
summed_df = merged_df.groupby('user', as_index=False).sum()
summed_df.to_csv("summed_data.csv", index=False)

In [ ]:
import os

folder_path = "C:\\Users\\tony9\\Downloads\\rplacemergedcondensed"

file_names = ['filename.csv', 'filename1.csv', 'filename3.csv', 'filename4.csv']

df_list = []
for file in file_names:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    df_list.append(df)

merged_df = pd.concat(df_list)

grouped_df = merged_df.groupby('user', as_index=False).sum()

output_file = os.path.join(folder_path, "merged_output.csv")
grouped_df.to_csv(output_file, index=False)

print("Merged file created at:", output_file)

In [ ]:
merged_condensed = pd.read_csv(r"C:\Users\tony9\Downloads\rplacemergedcondensed\merged_output.csv")
grouped_df = merged_condensed.groupby('user')['frequency'].sum().reset_index()
grouped_df.to_csv("C:/Users/tony9/Downloads/filename4.csv", index=False)

In [ ]:
import pandas as pd
from datetime import datetime
import os
from plotnine import *

# Testing code for loop

data = pd.read_csv(r"C:\Users\tony9\Downloads\2023_place_canvas_history-000000000000.csv")

data = data.drop(['coordinate', 'pixel_color'], axis=1)

def parse_timestamp(timestamp): # Check if the timestamp has microsecond part 
    if '.' in timestamp: 
        return datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S.%f %Z") 
    else: 
        return datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S %Z") 
    
data["timestamp"] = pd.to_datetime(data["timestamp"].apply(parse_timestamp))

data["day"] = data["timestamp"].dt.day
data["hour"] = data["timestamp"].dt.hour

data = data.drop(['timestamp'], axis=1)


unique_users_count = data.groupby(['day', 'hour'])['user'].nunique()

unique_users_day_hour = unique_users_count.reset_index()

# Rename the column for clarity
unique_users_day_hour.rename(columns={'user': 'unique_users_count'}, inplace=True)

unique_users_day_hour.to_csv(r"C:\Users\tony9\Downloads\rplace1.csv", index=False)

# Running loop on all data

file_paths = [f"C:\\Users\\tony9\\Downloads\\2023_place_canvas_history-{i:012d}.csv" for i in range(52)]

all_data = []

# Process each file and aggregate data
for file_path in file_paths:
    if os.path.exists(file_path):
        data = pd.read_csv(file_path)
        data = data.drop(['coordinate', 'pixel_color'], axis=1)
        data["timestamp"] = pd.to_datetime(data["timestamp"].apply(parse_timestamp))
        data["day"] = data["timestamp"].dt.day
        data["hour"] = data["timestamp"].dt.hour
        data = data.drop(['timestamp'], axis=1)
        unique_users_count = data.groupby(['day', 'hour'])['user'].nunique().reset_index()
        unique_users_count.rename(columns={'user': 'unique_users_count'}, inplace=True)
        all_data.append(unique_users_count)
    else:
        print(f"File not found: {file_path}")

combined_data = pd.concat(all_data)

final_data = combined_data.groupby(['day', 'hour'])['unique_users_count'].sum().reset_index()

final_data.to_csv(r"C:\Users\tony9\Downloads\aggregated_data.csv", index=False)

data = pd.read_csv(r"C:\Users\tony9\Downloads\aggregated_data.csv")

# Correcting the code to create a bar chart for average daily users using plotnine
plot = (ggplot(data=data, mapping=aes(x='day', y='unique_users_count'))
                                       + geom_bar(stat="identity", fill="skyblue")
                                       + labs(title="Average Daily Users (Bar Chart)", x="Day", y="Average Users Count")
                                       + ylab("Daily Unique Users")
                                       + theme(figure_size=(12, 6))
                                      )

plot


data = pd.read_csv(r"C:\Users\tony9\Downloads\2023_place_canvas_history-000000000000.csv")

data = data.drop(['coordinate', 'user'], axis=1)
data["timestamp"] = pd.to_datetime(data["timestamp"].apply(parse_timestamp))
data["day"] = data["timestamp"].dt.day
data["hour"] = data["timestamp"].dt.hour
data = data.drop(['timestamp'], axis=1)

# Calculate the total pixels placed for each hour and day combo
total_pixels_placed = data.groupby(['day', 'hour']).size().reset_index()
total_pixels_placed.rename(columns={0: 'total_pixels_placed'}, inplace=True)

total_pixels_placed.to_csv(r"C:\Users\tony9\Downloads\rplace69.csv", index=False)

# Running loop on all data
file_paths = [f"C:\\Users\\tony9\\Downloads\\2023_place_canvas_history-{i:012d}.csv" for i in range(52)]
all_data = []

for file_path in file_paths:
    if os.path.exists(file_path):
        data = pd.read_csv(file_path)
        data = data.drop(['coordinate', 'user'], axis=1)
        data["timestamp"] = pd.to_datetime(data["timestamp"].apply(parse_timestamp))
        data["day"] = data["timestamp"].dt.day
        data["hour"] = data["timestamp"].dt.hour
        data = data.drop(['timestamp'], axis=1)
        total_pixels_placed = data.groupby(['day', 'hour']).size().reset_index()
        total_pixels_placed.rename(columns={0: 'total_pixels_placed'}, inplace=True)
        all_data.append(total_pixels_placed)
    else:
        print(f"File not found: {file_path}")

# Combine all data
combined_data = pd.concat(all_data)

final_data = combined_data.groupby(['day', 'hour'])['total_pixels_placed'].sum().reset_index()

final_data.to_csv(r"C:\Users\tony9\Downloads\aggregated_data200.csv", index=False)

In [ ]:
data = pd.read_csv(r"C:\Users\tony9\Downloads\aggregated_data.csv")

In [ ]:
plot = (
    ggplot(data, aes(x='factor(day)', y='unique_users_count')) +
    geom_boxplot(fill = 'skyblue') +
    theme(
        axis_text_x=element_text(angle=90),
        figure_size=(10, 6)
    ) +
    labs(
        title='Distribution of Unique Users by Day',
        x='Day',
        y='Unique Users'
    ) +
    theme_minimal()
)

plot

In [ ]:
# Grouping data by hour and calculating the average number of unique users per hour
hourly_average = data.groupby('hour')['unique_users_count'].mean().reset_index()

# Creating a bar plot using ggplot from plotnine
plot = (
    ggplot(hourly_average, aes(x='hour', y='unique_users_count')) +
    geom_bar(stat='identity', fill='skyblue') +
    theme(axis_text_x=element_text(angle=90), figure_size=(10, 6)) +
    labs(
        title='Average Number of Users by Hour',
        x='Hour',
        y='Unique Users'
    )
    + theme_minimal()
)

# Displaying the plot
plot

In [ ]:
data_200 = pd.read_csv(r"C:\Users\tony9\Downloads\aggregated_data200.csv")
merged_data = pd.merge(data, data_200, on=['day', 'hour'])

In [ ]:
plot = (
    ggplot(merged_data, aes(x='unique_users_count', y='total_pixels_placed')) +
    geom_point(color='blue', alpha=0.5) +
    labs(title='Scatter Plot of Unique Users vs Total Pixels Placed',
         x='Unique Users Count',
         y='Total Pixels Placed') +
    theme_minimal() +
    theme(figure_size=(10, 6))
)

plot

In [ ]:
hourly_average = data_200.groupby('hour')['total_pixels_placed'].mean().reset_index()

plot = (
    ggplot(hourly_average, aes(x='factor(hour)', y='total_pixels_placed')) +
    geom_bar(stat='identity', fill='skyblue') +  # Using geom_bar for bar chart
    theme(axis_text_x=element_text(angle=90), figure_size=(10, 6)) +
    labs(
        title='Average Tile Placements by Hour',
        x='Hour',
        y='Average Placements'
    )
    + theme_minimal()
)

plot

In [ ]:
# this data comes from previously merged data files in above code. Only unique users & respective frequency columns
data = pd.read_csv(r"C:\Users\tony9\Downloads\rplacemergedcondensed\merged_output.csv")

count_high_frequency = data[data['frequency'] >= 5700].shape[0]
# finding max frequency
print("Count of users with at least 21 in frequency:", count_high_frequency)

import matplotlib.pyplot as plt

bins = [0, 5, 10, 25, 50, 100, 200,500, 1000, 5700]
labels = ['0-5', '6-10','11-25', '26-50', '51-100', '101-200', '201-500','501-1000','1001-5700']
data['frequency_range'] = pd.cut(data['frequency'], bins=bins, labels=labels, right=False)

# Count users in each range
frequency_distribution = data['frequency_range'].value_counts()

# adjusting pie chart output
explode_values = [0.05] * len(frequency_distribution)
plt.figure(figsize=(10, 10))
plt.pie(frequency_distribution, labels=None, autopct='%1.1f%%', pctdistance=0.85, explode=explode_values)
plt.legend(frequency_distribution.index, title="Slice Numbers", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
plt.title('Pixel Frequency', fontsize = 20)

plt.show()

## Jake Code

In [ ]:
data = pd.read_csv('merged-csv-condensed-files.csv')

In [ ]:
# Sanity Check
len(data[data['frequency'] == 1])/len(data)
len(data[data['frequency'] == 20])/len(data)

In [ ]:
#Average number of pixels placed
data['frequency'].mean()

In [ ]:
#For loop to find the proportion of users who placed a certain amount of pixels. Only did the first 20 for simplicity's sake.
df = pd.DataFrame(columns=['Frequency', 'Proportion'])
for i in np.arange(1, 21):
    x = len(data[data['frequency'] == i])/len(data)
    df = df.append({'Frequency': i, 'Proportion': x}, ignore_index=True)

print(df)

In [ ]:
df.to_csv("props.csv")

## Tuukka Code

In [ ]:
data1 = pl.scan_parquet('<REPLACE WITH FILE PATH>', n_rows = 1000000)

In [ ]:
# Parse Coordinates
data1 = data1.with_columns(
    pl.col('coordinate').map_elements(parse_coordinate, return_dtype=pl.Object)
)
data1 = data1.with_columns([
    pl.col('coordinate').map_elements(lambda x: int(x.split(",")[0]), return_dtype=pl.Int32).alias('x'),
    pl.col('coordinate').map_elements(lambda x: int(x.split(",")[1]), return_dtype=pl.Int32).alias('y')
])

In [ ]:
# Some Polars stuff that Tuukka did
user_counts = data1.select("user").group_by("user").agg(pl.col('user').count().alias('count')).sort('count', descending = True)
user_counts = user_counts.with_columns(pl.col('count').cast(int))

user_counts = user_counts.filter(pl.col('count') > 20)

count_list = user_counts.select('user').collect(streaming = True)

data1 = data1.filter(pl.col('user').is_in(count_list))

user_counts.select(pl.len()).collect()

In [ ]:
# Some more Polars stuff that Tuukka did
sorted_ = data1.sort(pl.col(['user', 'timestamp']))

sorted_ = sorted_.with_columns(pl.col('timestamp').str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S%.f %Z").cast(pl.Datetime, strict = False))

sorted_ = sorted_.group_by('user').agg(pl.col('timestamp').diff(null_behavior = "drop").alias('diff'))

sorted_ = sorted_.explode('diff').drop_nulls()

sorted_std = sorted_.group_by('user').agg(pl.col('diff').std().alias('std'))

sorted_mean = sorted_.group_by('user').agg(pl.col('diff').mean().alias('mean'))

sorted_comb = sorted_std.join(sorted_mean, on = "user")

In [ ]:
# Join dataframes
data_join1 = data1.join(sorted_comb, on = "user")

data_join2 = data_join1.join(user_counts, on = "user")

In [ ]:
# Get min and max timestamp data
min_ = data_join2.select('timestamp').min().collect(streaming = True)

max_ = data_join2.select('timestamp').max().collect(streaming = True)

print(min_)
print(max_)

In [ ]:
# Time std plot
(ggplot(data_join2.collect(streaming=True)) +
    geom_point(aes(x = 'std', y = 'count')) +
    theme(
        figure_size = (16,9)
    )
)

In [ ]:
# Time mean plot
(ggplot(data_join2.collect(streaming=True)) +
    geom_point(aes(x = 'mean', y = 'count')) +
    theme(
        figure_size = (16,9)
    )
)

## Ben Code

In [ ]:
# Parquet File Writer
# Providing folder path
folder_path = "<REPLACE WITH FILE FOLDER PATH>"

# Partquet file name
parquet_file = "combined_place_data.parquet"

# Getting csv_file names
csv_files = glob.glob(os.path.join(folder_path, '**/*.csv'), recursive=True)

# Create Schema
schema = pa.schema([
    pa.field("timestamp", pa.string()),
    pa.field("user", pa.string()),
    pa.field("coordinate", pa.string()),
    pa.field("pixel_color", pa.string())
])

# Initializing writer
writer = parquet.ParquetWriter(parquet_file, schema, compression='snappy')

# Iterating through csv files
for file in sorted(csv_files):
    
    print("reading", file)
    table = csv.read_csv(file)
    # Casting table to schema
    table = table.cast(schema)

    print("writing", file)
    writer.write_table(table)
    
if writer:
    writer.close()
    
print(f"Combined CSVs to {parquet_file}")

In [ ]:
# Read the Parquet file and transform the data, maintain's lazyframe format
print("Read parquet, cast datetime, and parse coordinates")
df = pl.scan_parquet('combined_place_data.parquet')
df = df.with_columns(pl.col("timestamp").str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S%.f %Z').cast(pl.Datetime, strict=False))
df = df.with_columns(
    pl.col('coordinate').map_elements(parse_coordinate, return_dtype=pl.Object)
)
df = df.with_columns([
    pl.col('coordinate').map_elements(lambda x: int(x.split(",")[0]), return_dtype=pl.Int32).alias('x'),
    pl.col('coordinate').map_elements(lambda x: int(x.split(",")[1]), return_dtype=pl.Int32).alias('y')
])

In [ ]:
# Count number of pixels per unique user
value_counts = df.select("user").group_by("user").agg(pl.col('user').count().alias('count')).sort('count', descending=True)
value_counts = value_counts.with_columns(pl.col("count").cast(int))

In [ ]:
# Calculate pixels placed quantile
value_counts.select("count").quantile(quantile=.95).collect(streaming = True)

In [ ]:
# Index users with more than x number of pixels placed
pbs_idx = value_counts.filter(pl.col("count") >= 60)

# Select users and data from the prior list
pixels_placed = df.filter(pl.col('user').is_in(pbs_idx.select("user").collect(streaming=True))).collect(streaming=True)

# Filter down to top layer of pixels placed
latest_coords = pixels_placed.group_by(["x", "y"]).agg(
    pl.col('timestamp').arg_max().alias('idxmax'),
    pl.col("pixel_color").last().alias("latest_color")
)

# Write parquet file with pixel location and color
latest_coords.write_parquet("<REPLACE WITH FILE FOLDER PATH>.parquet")

In [ ]:
# Read in parquet file created above and sanity check
latest_coords = pd.read_parquet("latest_coords99.parquet")
latest_coords.head()

In [ ]:
# Create r/Place Image
plot = (ggplot(latest_coords, aes(x = "x", y = "y", color = "latest_color")) + 
    geom_point(shape = "s", size = .5) +
    theme(figure_size=[30, 20]) +
    scale_y_reverse() +
    scale_color_identity())

In [ ]:
# Draw Plot
plot.draw()

In [ ]:
# Save Plot
plot.save(limitsize=False, filename="<REPLACE WITH DESIRED FILE NAME AND DATATYPE>")